In [1]:
def return_now_runtime():
    import time
    now = time.localtime()
    return "%04d-%02d-%02d %02d-%02d-%02d" % (now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min, now.tm_sec)

## 1. 라이브러리 및 데이터
### Library & Data

In [2]:
import pandas as pd                         # 데이터 분석 라이브러리
import numpy as np                          # 계산 라이브러리
from tqdm import tqdm                       # 진행바
from sklearn.metrics import roc_auc_score   # AUC 스코어 계산
from sklearn.model_selection import KFold   # K-fold CV    
from bayes_opt import BayesianOptimization  # 베이지안 최적화 라이브러리  
from functools import partial               # 함수 변수 고정
import xgboost as xgb
import warnings
import joblib
warnings.filterwarnings("ignore")           # 경고 문구 미표시
model_name = 'xgboost'
%matplotlib inline

In [3]:
# train_path = 'data/train.csv'
# test_path = 'data/test.csv'
model_path = 'model/'+model_name+'_model.pkl'

## 2. 데이터 전처리
### Data Cleansing & Pre-Processing

In [4]:
# 입력하세요.

## 3. 탐색적 자료분석
### Exploratory Data Analysis

In [5]:
# 입력하세요.

## 4. 변수 선택 및 모델 구축
### Feature Engineering & Initial Modeling

1) Feature로 각 플레이어의 종족, 게임 내 이벤트 발생 숫자, 플레이어간 이벤트 발생 숫자의 차이를 사용합니다.
1-1) 종족을 숫자로 변환 (T:0, P:1, Z:2)
1-2) 이벤트 발생 숫자는 8개 이벤트에 대해 각각 추출
1-3) 플레이어간 이벤트 발생 숫자를 Feature로 추가
1-4) 플레이어간 이벤트 발생 숫자 차이를 Feature로 추가

2) 모델로 XGBClassifier를 사용합니다.

3) 훈련 세트를 5등분 하여 5-fold Cross Validation을 수행합니다.

4) Bayesian optimization을 사용하여 하이퍼 파라미터를 조정합니다.
4-1) 조정되는 하이퍼 파라미터: -
4-2) Score로 AUC를 사용하며 AUC가 가장 높도록 하이퍼 파라미터 조정

In [6]:
def species_converter(string):
    if string == 'T':
        return 0
    elif string == 'P':
        return 1
    elif string == 'Z':
        return 2
    else:
        raise ValueError

def data_preparation(df, answer=False):
    game_ids = df['game_id'].unique()
    events = ['Ability', 'AddToControlGroup', 'Camera', 'ControlGroup', 'GetControlGroup', 'Right Click', 'Selection', 'SetControlGroup']
    unique_event_0, unique_event_1, delta_event = {}, {}, {}
    for event in events:
        unique_event_0['P0_' + event] = 0
        unique_event_1['P1_' + event] = 0
        delta_event['delta_' + event] = 0
        
    species = df.groupby(['game_id', 'player']).species.unique()
    event_count = df.groupby(['game_id', 'player']).event.value_counts()
    if answer:
        winners = df.groupby(['game_id']).winner.max()
    
    x_data, y_data = [], []
    for game_id in tqdm(game_ids):
        df_event_count = event_count[game_id].unstack(level=-1)
        df = pd.DataFrame(species[game_id])
        df = pd.concat([df, df_event_count], axis=1)   
        df = df.fillna(0)
        
        df_P0_species = pd.DataFrame([species_converter(df.loc[0]['species'][0])], columns=['P0_species'])        
        df_P1_species = pd.DataFrame([species_converter(df.loc[1]['species'][0])], columns=['P1_species'])
        df = df.drop(['species'], axis=1)

        df_P0_event = unique_event_0.copy()
        for column in df.columns:
            df_P0_event['P0_' + column] = df.loc[0][column]
        df_P0_event = pd.DataFrame(pd.Series(df_P0_event)).T

        df_P1_event = unique_event_1.copy()
        for column in df.columns:
            df_P1_event['P1_' + column] = df.loc[1][column]
        df_P1_event = pd.DataFrame(pd.Series(df_P1_event)).T
        
        df_delta_event = delta_event.copy()
        for column in df.columns:
            df_delta_event['delta_' + column] = df_P0_event['P0_' + column][0] - df_P1_event['P1_' + column][0]
        df_delta_event = pd.DataFrame(pd.Series(df_delta_event)).T

        out = pd.concat([df_P0_species, df_P0_event, df_P1_species, df_P1_event, df_delta_event], axis=1)
        out.index = [game_id]
        out.index.name = 'game_id'
        
        x_data.append(out)
        if answer:
            y_data.append(winners[game_id])  

    x_data = pd.concat(x_data)
    y_data = np.array(y_data)
    
    return x_data, y_data

In [7]:
# train.csv 원본 파일 읽고 data_preparation 진행
# train = pd.read_csv('data/train.csv')
# x_train, y_train = data_preparation(train, answer=True)

# 전처리된 데이터 csv 파일로 저장
# x_train.to_csv('data/baseline_x_train.csv')
# np.savetxt("data/baseline_y_train.csv", y_train, delimiter=",")
# x_train.head()

In [8]:
# 전처리된 csv 파일 읽기
x_train = pd.read_csv('data/baseline_x_train.csv', index_col='game_id')
y_train = np.loadtxt('data/baseline_y_train.csv', delimiter=',', dtype=np.float32)

In [9]:
def xgb_cv(learning_rate, n_estimators, max_depth, min_child_weight, gamma, subsample, colsample_bytree, reg_alpha, reg_lambda, objective='binary:logistic', tree_method='gpu_hist', predictor='cpu_predictor', x_data=None, y_data=None, n_splits=5, output='score'):
    score = 0
    kf = KFold(n_splits=n_splits)
    models = []
    for train_index, valid_index in kf.split(x_data):
        x_train, y_train = x_data.iloc[train_index], y_data[train_index]
        x_valid, y_valid = x_data.iloc[valid_index], y_data[valid_index]
        
        model = xgb.XGBClassifier(
            learning_rate =learning_rate,
            n_estimators=int(n_estimators),
            max_depth=int(max_depth),
            min_child_weight=min_child_weight,
            gamma=gamma,
            subsample=np.clip(subsample, 0, 1),
            colsample_bytree=np.clip(colsample_bytree, 0, 1),
            reg_alpha=reg_alpha,
            reg_lambda=reg_lambda,
            objective= objective,
#             tree_method=tree_method,
            predictor=predictor,
        )
        
        model.fit(x_train, y_train)
        models.append(model)
        
        pred = model.predict_proba(x_valid)[:, 1]
        true = y_valid
        score += roc_auc_score(true, pred)/n_splits
    
    if output == 'score':
        return score
    if output == 'model':
        return models

In [10]:
func_fixed = partial(xgb_cv, x_data=x_train, y_data=y_train, predictor='gpu_predictor', n_splits=5, output='score') 
xgbBO = BayesianOptimization(
    func_fixed, 
    {
        'learning_rate' : (0.0001, 0.1),
        'n_estimators': (16, 1024),
        'max_depth' : (4, 6),
        'min_child_weight' : (4, 6),
        'gamma' : (0, 1),
        'subsample' : (0, 1),
        'colsample_bytree' : (0, 1),
        'reg_alpha' : (0, 50), 
        'reg_lambda' : (0, 50),
    }, 
    random_state=4321
)
xgbBO.maximize(init_points=5, n_iter=30)

|   iter    |  target   | colsam... |   gamma   | learni... | max_depth | min_ch... | n_esti... | reg_alpha | reg_la... | subsample |
-------------------------------------------------------------------------------------------------------------------------------------
|  1        |  0.6288   |  0.0708   |  0.8151   |  0.07681  |  4.573    |  4.386    |  1.003e+0 |  20.31    |  37.89    |  0.08915  |
|  2        |  0.6452   |  0.3099   |  0.6189   |  0.04605  |  4.437    |  5.327    |  700.1    |  47.52    |  14.06    |  0.6199   |
|  3        |  0.639    |  0.3833   |  0.4004   |  0.09427  |  5.86     |  5.897    |  394.5    |  17.11    |  33.24    |  0.04232  |
|  4        |  0.6252   |  0.2322   |  0.4301   |  0.007886 |  5.533    |  5.708    |  167.6    |  5.057    |  13.55    |  0.03016  |
|  5        |  0.646    |  0.8378   |  0.5976   |  0.09312  |  4.991    |  4.753    |  261.7    |  30.09    |  16.48    |  0.4617   |
|  6        |  0.6258   |  0.3517   |  0.94     |  0.000796 | 

## 5. 모델 학습 및 검증
### Model Tuning & Evaluation
1) AUC가 가장 높은 하이퍼 파라미터를 사용해 최종 모델을 얻습니다.

2) 훈련 세트와 같은 방법으로 테스트 세트에서 Feature를 추출합니다.

3) 최종 모델을 사용해 예측을 수행합니다.

4) 예측 결과를 submission.csv로 저장합니다.

In [12]:
params = xgbBO.max['params']
models = xgb_cv(
    params['learning_rate'], 
    params['n_estimators'], 
    params['max_depth'], 
    params['min_child_weight'], 
    params['gamma'], 
    params['subsample'], 
    params['colsample_bytree'], 
    params['reg_alpha'], 
    params['reg_lambda'], 
    x_data=x_train, y_data=y_train, n_splits=5, output='model')
joblib.dump(models, model_path)

['model/xgboost_model.pkl']

In [13]:
# test.csv 원본 파일 읽고 data_preparation 진행
# test = pd.read_csv('data/test.csv')
# x_test, _ = data_preparation(test, answer=False)

# 전처리된 데이터 csv 파일로 저장
# x_test.to_csv('data/baseline_x_test.csv')
# x_test.head()

In [14]:
# 전처리된 csv 파일 읽기
x_test = pd.read_csv('data/baseline_x_test.csv', index_col='game_id')

In [15]:
models = joblib.load(model_path)
preds = []
for model in models:
    pred = model.predict_proba(x_test)[:, 1]
    preds.append(pred)
pred = np.mean(preds, axis=0)

submission = pd.read_csv('data/sample_submission.csv', index_col=0)
submission['winner'] = submission['winner'] + pred
submission.to_csv(f'submission_{model_name}_{return_now_runtime()}.csv')
submission.head()

,winner
game_id,
38872,0.615644
38873,0.491244
38874,0.419966
38875,0.304518
38876,0.480850


## 6. 결과 및 결언
### Conclusion & Discussion

In [ ]:
# 입력하세요.